# Анализируем полученные данные. Строим графики, предсказываем вакансии...

In [1]:
import pickle
from collections import *
import collections
import numpy as np
import networkx as nx
from tqdm import tqdm
import json
from typing import List, Tuple, Sequence
import pickle
from tqdm import tqdm_notebook as tqdm
import pymorphy2
import nltk
from cytoolz import pipe
from collections import Counter
import re
import theano
from collections import defaultdict
from collections import Counter

In [2]:
# 04:10 утра 15 января, отключаем ворнинги
import warnings
warnings.filterwarnings("ignore")

Такие же функции, что и в data_parse, ничего интересного

In [3]:
morph = pymorphy2.MorphAnalyzer()
stopwords = nltk.corpus.stopwords.words(
    'russian') + nltk.corpus.stopwords.words('english')
stopwords += [
    'отличный', 'метр', 'наш', 'клиент', 'банка', 'проект', 'литр',
    'желательный', 'др', 'самый', 'мочь', 'хороший', 'год', 'чел', 'обязательный', 'опыт', 'работа',
    'качества', 'работа', 'свои'
]

cache = {}


def remove_numbers(text):
    return re.sub(r'\d+', '', text)


def _get_POS(word):
    if word not in cache:
        cache[word] = morph.parse(word)[0]
    return cache[word].tag.POS


def normal_form(word):
    if word not in cache:
        cache[word] = morph.parse(word)[0]
    return cache[word].normal_form


def is_word_pos_in(word: str, pos: List[str] = None) -> bool:
    if not pos:
        pos = ['NOUN', "ADJF", 'INFN', 'VERB', 'ADJS']

    return _get_POS(word) in pos


def get_words(text):
    return re.findall(r'\w+', text)


def nonempty(x):
    if isinstance(x, Sequence):
        return filter(lambda x: len(x) > 0 and x != ' ', x)
    return x


helper = {}


def remove_numbers(text):
    return re.sub(r'\d+', '', text)


def normalize_skill(skill: str):
    parsed = tuple(
        pipe(
            skill,
            lambda x: x.lower(),
            remove_numbers,
            get_words,
        ))

    clear_skill = []
    dirty_skill = []

    # Последнее стоп слово для dirty_skill
    last_stopword = None

    # Для каждого слова в скилле
    for i in parsed:
        # Нормализуем слово
        word = normal_form(i)

        # Если стоп слово - запомним его
        if word in nltk.corpus.stopwords.words('russian'):
            last_stopword = word

            if word == "без":
                clear_skill.append(word)

        # Проверим на часть речи, длинну и стоплова
        elif is_word_pos_in(word) and len(word) > 3 and word not in stopwords:

            # Если до этого было стоп слово, добавим его в dirty
            if last_stopword and len(dirty_skill) > 0:
                dirty_skill.append(last_stopword)
                last_stopword = None

            # Добавим в чистый скилл слово
            clear_skill.append(word)

            if is_word_pos_in(word, ['NOUN', 'ADJF']):
                dirty_skill.append(i)

    if len(dirty_skill) > 2 and len(dirty_skill) < 8:
        return clear_skill, dirty_skill

    else:
        return []

In [4]:
sim_to_work = pickle.load(open("sim_to_work.pck", "rb"))

In [5]:
vacancy_by_name = pickle.load(open("vacancy_by_name.pck", "rb"))

In [6]:
vacancy_by_name['e93e7dc6aac1811f5d9e0e872665f74b6d1e9c02']

{'candidat': 'Опыт работы от 3 лет, Знание складской и транспортной логистики,\nуверенный пользователь ПК, 1С\nЖелательно проживание в районе МО, г. Одинцово',
 'clear_skills': ['опыт работа',
  'знание транспортный логистика',
  'знание складской',
  'уверенный пользователь',
  'проживание район'],
 'dirty_skills': ['опыт работы',
  'знание транспортной логистики',
  'знание складской',
  'уверенный пользователь',
  'проживание в районе'],
 'drive_license': '',
 'education_value': 'неполное высшее',
 'employer_type_value': 'прямой работодатель',
 'experience_value': 'от 3 до 6 лет',
 'id': 'e93e7dc6aac1811f5d9e0e872665f74b6d1e9c02',
 'id_client': 'b2fe7d405fb9e1ab3f06656c244e163def331c75',
 'languages': [],
 'metro': [],
 'payment_from': 45000,
 'payment_to': 0,
 'place_of_work_value': 'на территории работодателя',
 'profession': 'Кладовщик',
 'profession_tree_id': 301,
 'profession_tree_name': 'Кладовщик',
 'town': 'Москва',
 'type_of_work_value': 'полный рабочий день',
 'vac': array

In [7]:
resume = pickle.load(open("resume_with_days.pkl", "rb"))

In [8]:
skills_by_theme = collections.defaultdict(lambda: [])

for vac in vacancy_by_name:
    vac = vacancy_by_name[vac]
    skills_by_theme[vac['profession_tree_name']].extend(vac['dirty_skills'])

In [9]:
for theme in tqdm(skills_by_theme):
    skills_by_theme[theme] = [i[0] for i in collections.Counter(skills_by_theme[theme]).most_common(6000)]

A Jupyter Widget

In [10]:
def get_jobs(resume_id):
    '''Отсортируем работы по времени. 1999, 2000, 2010'''
    ans = {}
    for job in resume[resume_id]['work_history']:
        ans[int(job['monthbeg'])/12 + int(job['yearbeg'])] = job
    return dict(collections.OrderedDict(sorted(ans.items())))

In [11]:
an = pickle.load(open("tmp.pck", "rb"))

In [12]:
skills_by_theme = collections.defaultdict(lambda: [])

for vac in vacancy_by_name:
    vac = vacancy_by_name[vac]
    skills_by_theme[vac['profession_tree_name']].extend(vac['clear_skills'])

In [13]:
for skill in skills_by_theme:
    skills_by_theme[skill] = Counter(skills_by_theme[skill]).most_common(10)

In [14]:
def get_skills(_id):
    '''Вытаскиваем навыки данной работы'''
    if _id in sim_to_work:
        sim = sim_to_work[_id]
        work = vacancy_by_name[sim[0][0]]['profession_tree_name']
        print("Название работы:", work)
        return skills_by_theme[work]
    else:
        return [[None]]

# Пошагово посмотрим на то, как добавляются навыки
Посмотрим на навыки, которые добавляются с каждой вакансией и на название вакансии...

In [15]:
user = 0
now = get_jobs(user)
old_skills = set()

for key in now.keys():
    tmp = set([i[0] for i in get_skills(now[key]['id'])])
    tmp -= old_skills
    old_skills |= tmp
                     
    print("Новые навыки:", tmp, end="\n\n")
    
print("Итоговые скиллы:")
print(old_skills)

Название работы: Промоутер
Новые навыки: {'коммуникативный навык', 'желание работать', 'умение общаться', 'желание общаться человек', 'желание зарабатывать', 'опрятный внешний', 'активный жизненный позиция', 'приятный внешность', 'грамотный речь', 'позитивный настрой'}

Название работы: Кладовщик
Новые навыки: {'опыт работа кладовщик', 'образование среднее специальный', 'уверенный пользователь', 'опыт работа склад', 'опыт работа аналогичный', 'знание программа', 'умение работать команда', 'опыт работа', 'опыт работа аналогичный должность'}

Название работы: Торговый представитель
Новые навыки: {'опыт работа торговый представитель', 'наличие автомобиль', 'опыт работа продажа', 'нацеленность результат', 'наличие личный автомобиль', 'водительский право категория'}

Название работы: Экономист аналитик
Новые навыки: {'сводный таблица', 'высокий образование', 'высокий образование экономический', 'опыт работа экономист', 'высокий экономический образование', 'аналитический склад'}

Название ра

In [16]:
def career(jobs):
    '''Текущая карьера по работе'''
    jobs = get_jobs(jobs)
    ans = list()
    for key in jobs:
        _id = jobs[key]['id']
        if _id in sim_to_work:
            ans.append(vacancy_by_name[sim_to_work[_id][0][0]]['profession_tree_name'])
            
    return ans

In [17]:
from joblib import delayed, Parallel

def parallel(f, data):
    """Run parallel your func on all CPU"""
    
    return Parallel(n_jobs=-1, verbose=3, max_nbytes='1G')(delayed(f)(x) for x in data)

In [18]:
data = parallel(career, range(len(resume)))

[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 21944 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 65784 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 128945 out of 128945 | elapsed:   16.4s finished


In [19]:
data.sort(key=len)

In [20]:
normal_data = []

for i in data[::-1]:
    normal_data.append(i)

In [21]:
# Матри
matrix = defaultdict(lambda: Counter())

for story in normal_data:
    if len(story) > 0:
        old_line = story[0]
        for line in story[1:]:
            matrix[line][old_line] += 1
            old_line = line

In [22]:
# Наш коэф. на основе того, что обычно встречается после и до tree_name
# Директор = 10
# Водитель = 3
# Промоутер = 1
k = pickle.load(open("p.pck", 'rb'))

In [ ]:
print(k)

In [ ]:
answer = []

for story in normal_data:
    last_ball = 0
    cool = True
    
    if len(set(story)) > 3:
        new_story = []
        
        for line in story:
            if not last_ball > k[line]:
                new_story.append(line)
                last_ball = k[line]
    
        if cool:
            answer.append(story)

# Построение графа от желаемой должности

In [ ]:
def get_all_ages(prof, used = None):
    '''Построение дерева до желаемой tree_name'''
    global k
    if not used:
        used = [prof]
    else:
        used.append(prof)
    
    now = []
    
    for i in matrix[prof].most_common(100):
        if i[0] not in used:

            if k[i[0]] < k[prof] and k[i[0]] + 2 >= k[prof] and k[i[0]] > 2:
                now.append(i[0])

                if len(now) == 2:
                    for j in now:
                        if prof and prof not in a:
                            a.add_node(prof)
                            
                        if j and j not in a:
                            a.add_node(j)
                            
                        if j:
                            a.add_edge(j, prof)
                        get_all_ages(j, used)
    


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
a = nx.DiGraph()
get_all_ages("Генеральный директор")
plt.figure(3,figsize=(12,12)) 
nx.draw(a, node_color='b', node_size=500, alpha=0.5, with_labels=True)
plt.show()

In [ ]:
matrix['Генеральный директор'].most_common(1)

In [ ]:
matrix['Финансовый директор'].most_common(3)

In [ ]:
matrix['Экономист аналитик'].most_common(3)

In [ ]:
matrix['Финансовый менеджер'].most_common(4)

In [ ]:
matrix['Помощник бухгалтера'].most_common(3)

In [ ]:
matrix['Оператор ПК'].most_common(3)

In [ ]:
normal_data

In [ ]:
matrix['Педагог']

In [ ]:
themes = defaultdict(lambda: [])
dirty_to_normal = {}

for d in vacancy_by_name:
    d = vacancy_by_name[d]
    themes[d['profession_tree_name']].append(d['dirty_skills'])

In [ ]:
def get_tfidf(theme):
    
    tmp = []

    for i in themes[theme]:
        tmp.extend(i)

    tf = Counter(tmp)

    for j in tf:
        tf[j] /= len(tmp)

    for word in tf:
        idf = np.log(len(themes[theme])/ sum([word in doc for doc in themes[theme]]))
        tf[word] *= idf

    return tf

In [ ]:
# Все tree_name, которые есть
themes.keys()

In [ ]:
# Самые чатые навыки Аудитора
get_tfidf('Аудитор').most_common(100)

In [ ]:
cahce = pickle.load(open("cahce.pck", "rb"))

In [ ]:
vacancy = []

for k in vacancy_by_name:
    vacancy.append(vacancy_by_name[k])

In [ ]:
def _clear(text):
    return list(filter(lambda x: x not in stopwords, get_words(remove_numbers(text.lower()))))

In [ ]:
def to_vec(tupl):
    skills_vecs = []
    for skill in tupl:
        skill_vec = np.sum(list(map(lambda x: cahce[x], _clear(skill))), axis=0)
        skills_vecs.append(skill_vec)
    return np.mean(skills_vecs, axis=0)

In [ ]:
len(vacancy_by_name)

In [ ]:
vacancy = pickle.load(open("vacancy_with_vecs.pck", "rb"))

In [ ]:
themes_vectors = defaultdict(lambda: [])

In [ ]:
for vac in vacancy:
    if not np.isnan(vac['vec']).all():
        themes_vectors[vac['profession_tree_name']].append(vac['vec'])

In [ ]:
for theme in themes_vectors:
    themes_vectors[theme] = np.sum(themes_vectors[theme], axis=0)

In [ ]:
def compile_cos_sim_theano():
    v1 = theano.tensor.vector(dtype='float32')
    v2 = theano.tensor.vector(dtype='float32')
    
    numerator = theano.tensor.sum(v1*v2)
    denominator = theano.tensor.sqrt(theano.tensor.sum(v1**2)*theano.tensor.sum(v2**2))
   
    return theano.function([v1, v2], numerator/denominator)

cos_sim_theano_fn = compile_cos_sim_theano()

In [ ]:
def get_theme(tupl):
    vec = to_vec(tupl)
    a = Counter()
    
    for theme in themes_vectors:
        a[theme] = cos_sim_theano_fn(themes_vectors[theme], vec)
    
    for u in a.most_common(10):
        if k[u[0]] > 2: 
            return u[0]

In [ ]:
k = pickle.load(open("p.pck", 'rb'))

In [ ]:
k['Техник']

In [ ]:
def draw_path(theme):
    '''Всё в одном.'''
    a = nx.DiGraph()

    def get_all_ages(prof, used = None):
        global k
        if not used:
            used = [prof]
        else:
            used.append(prof)

        now = []

        for i in matrix[prof].most_common(100):
            if i[0] not in used:

                if k[i[0]] < k[prof] and k[i[0]] + 2 >= k[prof] and k[i[0]] > 2:
                    now.append(i[0])

                    if len(now) == 2:
                        for j in now:
                            if prof and prof not in a:
                                a.add_node(prof)

                            if j and j not in a:
                                a.add_node(j)

                            if j:
                                a.add_edge(j, prof)
                            get_all_ages(j, used)

    get_all_ages(theme)
    plt.figure(3,figsize=(12,12)) 
    nx.draw(a, node_color='b', node_size=500, alpha=0.5, with_labels=True)
    plt.show()

In [ ]:
draw_path("Финансовый директор")

# Построение графа до желаемой должности

In [ ]:
# matrix[i][j] - кол-во человек, которое из должности i перешли в должности j
matrix_back = defaultdict(lambda: Counter())

for line in normal_data:
    if len(line) > 1:
        first = line[0::2]
        second = line[1::2]
        
        for i in range(len(second)):
            if k[first[i]] < k[second[i]]:
                matrix_back[first[i]][second[i]] += 1

In [ ]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [ ]:
api_key = 'Kyxdr2dZqCwvx2yfpTvd'
import plotly
plotly.tools.set_credentials_file(username='tvorogme', api_key=api_key)

In [ ]:
matrix_back['Финансовый директор']

In [ ]:
vacs_by_theme = defaultdict(lambda: [])

for vac in vacancy:
    vacs_by_theme[vac['profession_tree_name']].append(vac)

In [ ]:
sim_to_work = pickle.load(open("sim_to_work.pck", "rb"))

In [ ]:
for work in sim_to_work:
    if sim_to_work[work]:
        tmp = vacancy_by_name[sim_to_work[work][0][0]]

        if 'popularity' not in tmp:
            tmp['popularity'] = 0

        tmp['popularity'] += 1

In [ ]:
similar_to_theme = {}

for theme in tqdm(themes):
    cool = []
    for name in vacancy_by_name:
        tmp = vacancy_by_name[name]
        if 'popularity' in tmp and tmp['profession_tree_name'] == theme:
            cool.append([tmp['popularity'], tmp['id']])
    cool.sort(reverse=True)
    similar_to_theme[theme] = cool[:10]

In [ ]:
import gensim

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format("ruwikiruscorpora_upos_cbow_300_20_2017.bin.gz", binary=True)

In [ ]:
tmp_vector = np.array([0] * 300, dtype=np.float32)

def _word2vec(word):
    for i in ["_NOUN", "_ADJ", "_VERB"]:
        tmp = "{}{}".format(word, i)
        
        if tmp in model:
            return model[tmp]
        else:
            return tmp_vector

def to_vec(tupl):
    skills_vecs = []
    for skill in tupl:
        
        skill_vec = np.asarray(list(map(lambda x: _word2vec(morph.parse(x)[0].normal_form), skill)))
        if not np.array_equal(tmp_vector, skill_vec):
            skills_vecs.append(np.mean(skill_vec, axis=0))
    return np.mean(skills_vecs, axis=0)

In [ ]:
def recommend_vac(_next, cur_skills):
    '''Рекоммендуем вакансию по скиллам'''
    current_vec = to_vec(cur_skills)
    vecs = [to_vec(vacancy_by_name[i[1]]['clear_skills']) for i in similar_to_theme[_next]]

    vecs = list(filter(lambda x: not np.isnan(x).all(), vecs))

    sim = list(map(lambda x: float(cos_sim_theano_fn(x, current_vec)), vecs))
    vacs = [vacancy_by_name[i[1]]['profession'] for i in similar_to_theme[_next]]

    return Counter({x:y for x,y in zip(vacs, sim)}).most_common(3)

In [ ]:
recommend_vac("")

In [ ]:
cur_skills = ['грамотная речь', 'самостоятельного поиска информации'] 
_next = "Редактор"

In [ ]:
[i[0] for i in recommend_vac(_next, cur_skills)]

In [ ]:
from plotly.plotly import iplot
def draw(theme, cur_skills= None):
    a = nx.DiGraph()

    def get_all_ages(prof, used=None):
        global k
        if not used:
            used = [prof]
        else:
            used.append(prof)

        now = []

        # {'Генеральный директор': 19}
        for i in matrix_back[prof].most_common():
            if i[0] not in used:  # False
                rating_next = k[i[0]]  # 10
                rating_now = k[prof]  # 9

                if rating_next > rating_now and rating_now + 2 >= rating_next:
                    now.append(i[0])

                    if len(now) > 2:
                        break
        for j in now:
            if prof and prof not in a:
                a.add_node(prof)

            if j and j not in a:
                a.add_node(j)

            if j:
                a.add_edge(prof, j)

            get_all_ages(j, used)

    get_all_ages(theme)
    p = nx.random_layout(a)
    print(1)

    new_p = {}
    prev_level_count = defaultdict(lambda: 0)

    for node in a.nodes():
        # (Название, x, y)

        i = [0, 0]
        i[0] = k[node] + 0.03 * prev_level_count[k[node]]
        i[1] = prev_level_count[k[node]]
        prev_level_count[k[node]] += 1
        new_p[node] = i
    print(2)
    p = new_p
    
    def _arrow(x, y, x2, y2):
        return dict(
            ax=x2,
            ay=y2,
            axref='x',
            ayref='y',
            x=x,
            y=y,
            xref='x',
            yref='y',
            showarrow=True,
            arrowhead=2,
            arrowsize=2,
            arrowwidth=1,
            arrowcolor='#636363'
        )
    arrows = []
    for edge in a.edges():
        x0, y0 = p[edge[0]]
        x1, y1 = p[edge[1]]

        arrows.append(_arrow(x1, y1, x0, y0))
    print(3)
    node_trace = Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers',
        hoverinfo='text',
        showlegend = True,
        marker=Marker(
            showscale=True,
            reversescale=True,
            colorscale="YIGnBu",
            color=[],
            size=20))
    x_text, y_text, labels = [], [], []
    
    for node in a.nodes():
        x, y = p[node]
        x_text.append(x)
        y_text.append(y)
        
        node_trace['x'].append(x)
        node_trace['y'].append(y)
        
        if len(node.split()) > 1:
            a = node.split()
            a = a[:len(a)//2] + ["<br>"] + a[len(a)//2:]
            labels.append(" ".join(a))
        else:
            labels.append(node)
            
        if cur_skills:
            if node == theme:
                node_trace['text'].append(None)
            else:
                t = []
                for i in recommend_vac(node, cur_skills):
                    
                    if len(i[0].split()) > 1:
                        a = i[0].split()
                        a = a[:len(a)//2] + ["<br>"] + a[len(a)//2:]
                        t.append(" ".join(a))
                    else:
                        t.append(i[0])

                    
                
                node_trace['text'].append("<br><br>".join(t))
        
        if node == theme:
            node_trace['marker']['color'].append("red")
        else:
            node_trace['marker']['color'].append(k[node])
    print(4)
    trace2 = Scatter(
        x=x_text,
        y=y_text,
        mode='markers+text',
        name='Markers and Text',
        text=labels,
        textposition='top',
        hoverinfo='none',
        showlegend = False,
        marker=Marker(color=[],size=0)
    )

    data = [node_trace, trace2]
    layout = Layout(showlegend=False, annotations=arrows)
    fig = Figure(data=data, layout=layout)
    return fig

In [ ]:
init_notebook_mode(connected=True)

In [ ]:
iplot(draw("Журналист"))

# Нашы навыки

----

In [ ]:
our_skills = ['знание законодательства']
get_theme(our_skills)

In [ ]:
our_skills = ['грамотная речь', 
              'самостоятельного поиска информации'] 
get_theme(our_skills)

In [ ]:
our_skills = ['сохранением конфиденциальности информации']
get_theme(our_skills)

In [ ]:
our_skills = ['высшее профильное образование']
get_theme(our_skills)

In [ ]:
our_skills = ['опыт работы аудитором']
get_theme(our_skills)

----

# Какая карьрная лестница до аудитора?

В гитхабе не видно, но это выглядит так:

* эта штука интерактивная, подробности: .... *

In [ ]:
from IPython.display import Image
Image("latest-screenshot.png")

In [ ]:
draw_path("Аудитор")

# Ок, сервис, у меня таки навыки. Кем я буду дальше?

In [ ]:
our_skills = ['грамотная речь', 'организаторские способности']

In [ ]:
get_theme(our_skills)

In [ ]:
iplot(draw(get_theme(our_skills),  our_skills))

In [ ]:
get_theme(["желание зарабатывать", "ответственность"])

In [ ]:
def recommend_vac(_next, cur_skills_vec):
    '''Рекоммендуем вакансию по скиллам'''
    vecs = [to_vec(vacancy_by_name[i[1]]['clear_skills']) for i in similar_to_theme[_next]]

    vecs = list(filter(lambda x: not np.isnan(x).all(), vecs))

    sim = list(map(lambda x: float(cos_sim_theano_fn(x, cur_skills_vec)), vecs))
    vacs = [vacancy_by_name[i[1]]['profession'] for i in similar_to_theme[_next]]

    return Counter({x:y for x,y in zip(vacs, sim)}).most_common(3)

In [ ]:
from plotly.plotly import iplot
def draw(theme, cur_skills= None):
    a = nx.DiGraph()

    def get_all_ages(prof, used=None):
        global k
        if not used:
            used = [prof]
        else:
            used.append(prof)

        now = []

        # {'Генеральный директор': 19}
        for i in matrix_back[prof].most_common():
            if i[0] not in used:  # False
                rating_next = k[i[0]]  # 10
                rating_now = k[prof]  # 9

                if rating_next > rating_now and rating_now + 2 >= rating_next:
                    now.append(i[0])

                    if len(now) > 2:
                        break
        for j in now:
            if prof and prof not in a:
                a.add_node(prof)

            if j and j not in a:
                a.add_node(j)

            if j:
                a.add_edge(prof, j)

            get_all_ages(j, used)

    get_all_ages(theme)
    p = nx.random_layout(a)
    print(1)

    new_p = {}
    prev_level_count = defaultdict(lambda: 0)

    for node in a.nodes():
        # (Название, x, y)

        i = [0, 0]
        i[0] = k[node] + 0.03 * prev_level_count[k[node]]
        i[1] = prev_level_count[k[node]]
        prev_level_count[k[node]] += 1
        new_p[node] = i
    print(2)
    p = new_p
    
    def _arrow(x, y, x2, y2):
        return dict(
            ax=x2,
            ay=y2,
            axref='x',
            ayref='y',
            x=x,
            y=y,
            xref='x',
            yref='y',
            showarrow=True,
            arrowhead=2,
            arrowsize=2,
            arrowwidth=1,
            arrowcolor='#636363'
        )
    arrows = []
    for edge in a.edges():
        x0, y0 = p[edge[0]]
        x1, y1 = p[edge[1]]

        arrows.append(_arrow(x1, y1, x0, y0))
    print(3)
    node_trace = Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers',
        hoverinfo='text',
        showlegend = True,
        marker=Marker(
            showscale=True,
            reversescale=True,
            colorscale="YIGnBu",
            color=[],
            size=20))
    x_text, y_text, labels = [], [], []
    
    cur_vec = to_vec(cur_skills)
    
    for node in tqdm(a.nodes()):
        x, y = p[node]
        x_text.append(x)
        y_text.append(y)
        
        node_trace['x'].append(x)
        node_trace['y'].append(y)
        
        if len(node.split()) > 1:
            a = node.split()
            a = a[:len(a)//2] + ["<br>"] + a[len(a)//2:]
            labels.append(" ".join(a))
        else:
            labels.append(node)
            
        if cur_skills:
            if node == theme:
                node_trace['text'].append(None)
            else:
                t = []
                for i in recommend_vac(node, cur_vec):
                    
                    if len(i[0].split()) > 1:
                        a = i[0].split()
                        a = a[:len(a)//2] + ["<br>"] + a[len(a)//2:]
                        t.append(" ".join(a))
                    else:
                        t.append(i[0])

                    
                
                node_trace['text'].append("<br><br>".join(t))
        
        if node == theme:
            node_trace['marker']['color'].append("red")
        else:
            node_trace['marker']['color'].append(k[node])
    print(4)
    trace2 = Scatter(
        x=x_text,
        y=y_text,
        mode='markers+text',
        name='Markers and Text',
        text=labels,
        textposition='top',
        hoverinfo='none',
        showlegend = False,
        marker=Marker(color=[],size=0)
    )

    data = [node_trace, trace2]
    layout = Layout(showlegend=False, annotations=arrows)
    fig = Figure(data=data, layout=layout)
    return fig

In [ ]:
iplot(draw(get_theme(["желание зарабатывать", "ответственность"]),  our_skills))

In [ ]:
cur_skills = ['грамотная речь', 'самостоятельного поиска информации'] 
_next = "Редактор"
recommend_vac(_next, cur_skills)